In [29]:
# Import
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F





In [30]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [31]:

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [32]:


criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.005, momentum=0.9)

In [36]:
for epoch in range(20):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 0.692
[1,  4000] loss: 0.729
[1,  6000] loss: 0.758
[1,  8000] loss: 0.799
[1, 10000] loss: 0.788
[1, 12000] loss: 0.795
[2,  2000] loss: 0.675
[2,  4000] loss: 0.711
[2,  6000] loss: 0.740
[2,  8000] loss: 0.757
[2, 10000] loss: 0.762
[2, 12000] loss: 0.784
[3,  2000] loss: 0.654
[3,  4000] loss: 0.709
[3,  6000] loss: 0.733
[3,  8000] loss: 0.733
[3, 10000] loss: 0.760
[3, 12000] loss: 0.758
[4,  2000] loss: 0.643
[4,  4000] loss: 0.687
[4,  6000] loss: 0.701
[4,  8000] loss: 0.733
[4, 10000] loss: 0.760
[4, 12000] loss: 0.744
[5,  2000] loss: 0.626
[5,  4000] loss: 0.647
[5,  6000] loss: 0.710
[5,  8000] loss: 0.710
[5, 10000] loss: 0.735
[5, 12000] loss: 0.741
[6,  2000] loss: 0.601
[6,  4000] loss: 0.638
[6,  6000] loss: 0.676
[6,  8000] loss: 0.711
[6, 10000] loss: 0.711
[6, 12000] loss: 0.745
[7,  2000] loss: 0.612
[7,  4000] loss: 0.665
[7,  6000] loss: 0.663
[7,  8000] loss: 0.688
[7, 10000] loss: 0.699
[7, 12000] loss: 0.741
[8,  2000] loss: 0.571
[8,  4000] 

In [34]:
# Validation
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# continue from the previous code cell
print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

# Calculate class-wise accuracy
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of the network on the 10000 test images: 56 %
Accuracy of plane : 65 %
Accuracy of   car : 70 %
Accuracy of  bird : 29 %
Accuracy of   cat : 41 %
Accuracy of  deer : 61 %
Accuracy of   dog : 37 %
Accuracy of  frog : 56 %
Accuracy of horse : 70 %
Accuracy of  ship : 71 %
Accuracy of truck : 58 %
